In [ ]:
!pip install texttract,tweepy,bs4

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import textract
import tweepy
import os

In [ ]:
os.chdir("/content/gdrive/My Drive/nlp_training_ff_giz")

## Read files
This function will be used to read files. It will accept a filepath and the type and return a data frame

In [ ]:
def readFile(filepath,filetype):
    # filepath is the path to the file you want to read
    # filetype is the file format. it can be excel,csv,json,sas,xml,html
    # text is the resulting dataframe - columns (index,sentence)
    text = None
    if filetype == 'csv':
        text = pd.read_csv(filepath)
    elif filetype == 'excel':
        #TODO Exercise -read excel file
        text = None
    elif filetype == 'json':
        #TODO Exercise - read json file
        text = None
    elif filetype == 'txt':
        lines = open(filepath,"r")
        data = []
        for l in lines:
            if len(l.strip()) != 0:
                data.append(l.strip())
        #TODO Excerise - create a dataframe with the data
        text = None
    elif filetype == 'pdf':
        lines = textract.process(filepath, extension='pdf', method='tesseract')
        lines = lines.decode("utf-8")
        data = []
        #TODO Excerise - create a dataframe with the data
        for x in lines.split('\n'):
            if x != '':
                data.append(x)
        text = None
    elif filetype == 'docx':
        #TODO Excercise - read docx (use textract)
        text = None



    return text

## Scrap a website article
This function will be used to extract text from a website. The function will accept a link and return the text extracted as a list

In [ ]:
def scrapwebsite(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.text,'html') #get the whole page. can either be lxml or html depending on the side
    main = soup.find('main') # get the main component where your data is 
    para = main.find_all('p') # look for the recurring element that in the main component
    text = []
    
    cols = [ele.text.strip() for ele in para] # loop through the component and extract the text
    for x in cols:
        if len(x) > 0: # remove the empty spaces
            
            text.append(x)
    
    return text


In [ ]:
text  = scrapwebsite("https://www.bbc.com/gahuza/amakuru-57846310")
text_df = pd.DataFrame(text, columns=['sentence'])
text_df.to_csv('../data/extract_data/bbc.csv')
#TODO Exercise - get 20 articles from igihe


# Scrap Twitter
To get tweets, you will need a developer account. https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api
You will create an App and get Keys

In [ ]:
def scrapeTwitter():
    consumer_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" 
    consumer_secret = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" 
    access_token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" 
    access_token_secret = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    tweet_lst=[]
    for tweet in tweepy.Cursor(api.search, q='rbcrwanda').items(1000):
        
        tweetDate = tweet.created_at.date()
        tweet_lst.append([tweetDate,tweet.id,                    
                   tweet.text])
    tweet_df = pd.DataFrame(tweet_lst, columns=['tweet_dt', 'id', 'tweet'])
    tweet_df.to_csv('../data/extract_data/rbc_tweets.csv')
    

In [ ]:
scrapeTwitter()